In [2]:
import datetime

import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
def importdata():
    dataframe = pd.read_csv('../data/df_points_18-19-20-21_v1_weather_final.csv')
    return dataframe.sort_values(by=['dt_start'])

# Main

In [5]:
df = importdata()
df = df.reset_index(drop=True)
df = df.fillna(0)

df.drop(['time_start','time_end'], axis=1, inplace=True)


In [6]:
## Feature hex_id encoding
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
df['hex_enc']= label_encoder.fit_transform(df['hex_id'])

#del hex_id
del df['hex_id']

In [7]:
# Add start time Feature
df['start_min'] = np.nan
for ind in tqdm(df.index):
    df['start_min'][ind] = (df['dt_start'][ind] % 86400) / 60

  0%|                                               | 0/2435439 [00:00<?, ?it/s]/var/folders/5t/28jzvs016nq5vxslb670gs040000gn/T/ipykernel_16010/371371022.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['start_min'][ind] = (df['dt_start'][ind] % 86400) / 60
100%|███████████████████████████████| 2435439/2435439 [09:56<00:00, 4083.24it/s]


# Main

In [4]:
#df.to_csv('../data/df_points_18_21_v1_new_features.csv')
df = pd.read_csv('../data/df_points_18_21_v1_new_features.csv')

In [5]:
del df['Unnamed: 0']
df = df.sort_values(by='dt_start')
df.head()

,bike_id,lat,lng,timestood,temp,rain,snow,dt_start,dt_end,hex_enc,start_min
0,3694,51.323619,12.335887,12,10.80,0.0,0.0,1514764029,1514764799,324,1427.15
1,2508,51.323619,12.335887,12,10.80,0.0,0.0,1514764035,1514764799,324,1427.25
2,41851,51.332415,12.395158,2,10.80,0.0,0.0,1514764638,1514764799,137,1437.30
3,2508,51.323619,12.335887,1439,10.86,0.0,0.0,1514764800,1514851199,324,0.00
4,3694,51.323619,12.335887,821,10.86,0.0,0.0,1514764800,1514814103,324,0.00


In [6]:
TargetVariable=['timestood']
Predictors=['bike_id', 'lat', 'lng', 'temp', 'rain', 'snow', 'dt_start','hex_enc','start_min']

X=df[Predictors].values
y=df[TargetVariable].values

### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)
 
# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)
 
# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
 
# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1704807, 9)
(1704807, 1)
(730632, 9)
(730632, 1)


In [12]:
# importing the libraries
from keras.models import Sequential
from keras.layers import Dense

# create ANN model
model = Sequential()

# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=5, input_dim=9, kernel_initializer='normal', activation='relu'))

# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=5, kernel_initializer='normal', activation='tanh'))

# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

2022-03-02 13:10:43.476269: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
# Fitting the ANN to the Training set
model.fit(X_train, y_train ,batch_size = 15, epochs = 50, verbose=1)
 
# Generating Predictions on testing data
Predictions=model.predict(X_test)
 
# Scaling the predicted Price data back to original price scale
Predictions=TargetVarScalerFit.inverse_transform(Predictions)
 
# Scaling the y_test Price data back to original price scale
y_test_orig=TargetVarScalerFit.inverse_transform(y_test)
 
# Scaling the test data back to original scale
Test_Data=PredictorScalerFit.inverse_transform(X_test)
 
TestingData=pd.DataFrame(data=Test_Data, columns=Predictors)
TestingData['idel_time']=y_test_orig
TestingData['pred_idel_time']=Predictions
TestingData.head()

Epoch 1/50
113654/113654 [==============================] - 98s 858us/step - loss: 0.5205
Epoch 2/50
113654/113654 [==============================] - 99s 870us/step - loss: 0.5148
Epoch 3/50
113654/113654 [==============================] - 93s 818us/step - loss: 0.5138
Epoch 4/50
113654/113654 [==============================] - 89s 785us/step - loss: 0.5122
Epoch 5/50
113654/113654 [==============================] - 89s 781us/step - loss: 0.5110
Epoch 6/50
113654/113654 [==============================] - 89s 779us/step - loss: 0.5103
Epoch 7/50
113654/113654 [==============================] - 88s 777us/step - loss: 0.5096
Epoch 8/50
113654/113654 [==============================] - 89s 784us/step - loss: 0.5082
Epoch 9/50
113654/113654 [==============================] - 89s 783us/step - loss: 0.5071
Epoch 10/50
113654/113654 [==============================] - 89s 781us/step - loss: 0.5063
Epoch 11/50
113654/113654 [==============================] - 89s 784us/step - loss: 0.5051
Epoch 12

,bike_id,lat,lng,temp,rain,snow,dt_start,hex_enc,start_min,idel_time,pred_idel_time
0,41599.0,51.308364,12.379150,-2.99,0.0,0.0,1.616198e+09,98.0,0.000000,648.0,1065.932861
1,97541.0,51.326471,12.342482,21.45,0.0,0.0,1.594997e+09,329.0,888.600000,38.0,161.566406
2,41767.0,51.335989,12.326128,19.11,0.0,0.0,1.532736e+09,352.0,0.000000,1439.0,924.569458
3,93543.0,51.359000,12.376219,8.36,0.0,0.0,1.587600e+09,261.0,0.000000,268.0,1033.426270
4,97714.0,51.320307,12.425492,10.08,0.0,0.0,1.586595e+09,110.0,537.266667,902.0,290.292206


In [14]:
# Computing the absolute percent error
APE=100*(abs(TestingData['idel_time']-TestingData['pred_idel_time'])/TestingData['idel_time'])
TestingData['APE']=APE
 
print('The Accuracy of ANN model is:', 100-np.mean(APE))
TestingData.head()

The Accuracy of ANN model is: -inf


,bike_id,lat,lng,temp,rain,snow,dt_start,hex_enc,start_min,idel_time,pred_idel_time,APE
0,41599.0,51.308364,12.379150,-2.99,0.0,0.0,1.616198e+09,98.0,0.000000,648.0,1065.932861,64.495812
1,97541.0,51.326471,12.342482,21.45,0.0,0.0,1.594997e+09,329.0,888.600000,38.0,161.566406,325.174753
2,41767.0,51.335989,12.326128,19.11,0.0,0.0,1.532736e+09,352.0,0.000000,1439.0,924.569458,35.749169
3,93543.0,51.359000,12.376219,8.36,0.0,0.0,1.587600e+09,261.0,0.000000,268.0,1033.426270,285.606817
4,97714.0,51.320307,12.425492,10.08,0.0,0.0,1.586595e+09,110.0,537.266667,902.0,290.292206,67.816829


In [ ]:
# Function to generate Deep ANN model 
def make_regression_ann(Optimizer_trial):
    from keras.models import Sequential
    from keras.layers import Dense
    
    model = Sequential()
    model.add(Dense(units=5, input_dim=9, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer=Optimizer_trial)
    return model

###########################################
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

# Listing all the parameters to try
Parameter_Trials={'batch_size':[10,20,30],
                      'epochs':[10,20],
                    'Optimizer_trial':['adam', 'rmsprop']
                 }

# Creating the regression ANN model
RegModel=KerasRegressor(make_regression_ann, verbose=1)

###########################################
from sklearn.metrics import make_scorer

# Defining a custom function to calculate accuracy
def Accuracy_Score(orig,pred):
    MAPE = np.mean(100 * (np.abs(orig-pred)/orig))
    print('#'*70,'Accuracy:', 100-MAPE)
    return(100-MAPE)

custom_Scoring=make_scorer(Accuracy_Score, greater_is_better=True)

#########################################
# Creating the Grid search space
# See different scoring methods by using sklearn.metrics.SCORERS.keys()
grid_search=GridSearchCV(estimator=RegModel, 
                         param_grid=Parameter_Trials, 
                         scoring=custom_Scoring, 
                         cv=5)

#########################################
# Measuring how much time it took to find the best params
import time
StartTime=time.time()

# Running Grid Search for different paramenters
grid_search.fit(X,y, verbose=1)

EndTime=time.time()
print("########## Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes')

print('### Printing Best parameters ###')
grid_search.best_params_

/var/folders/5t/28jzvs016nq5vxslb670gs040000gn/T/ipykernel_17631/1882582903.py:24: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  RegModel=KerasRegressor(make_regression_ann, verbose=1)


Epoch 1/10
194836/194836 [==============================] - 152s 776us/step - loss: 0.4980
Epoch 2/10
194836/194836 [==============================] - 152s 781us/step - loss: 0.4937
Epoch 3/10
194836/194836 [==============================] - 152s 782us/step - loss: 0.4924
Epoch 4/10
194836/194836 [==============================] - 151s 776us/step - loss: 0.4912
Epoch 5/10
194836/194836 [==============================] - 151s 777us/step - loss: 0.4878
Epoch 6/10
194836/194836 [==============================] - 152s 780us/step - loss: 0.4840
Epoch 7/10
194836/194836 [==============================] - 151s 777us/step - loss: 0.4828
Epoch 8/10
194836/194836 [==============================] - 151s 777us/step - loss: 0.4817
Epoch 9/10
194836/194836 [==============================] - 153s 784us/step - loss: 0.4795
Epoch 10/10
48709/48709 [==============================] - 28s 581us/step
